**CosyVoice**

In [1]:
import sys
sys.path.append('./third_party/CosyVoice/')

from cosyvoice.cli.cosyvoice import CosyVoice
from cosyvoice.utils.file_utils import load_wav
import torchaudio

2024-08-09 11:23:03,199 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2024-08-09 11:23:03,201 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-08-09 11:23:03,268 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 750810b2d0c33f9be9cca0301d47437a and a total number of 980 components indexed


transformer is not installed, please install it if you want to use related modules
failed to import ttsfrd, use WeTextProcessing instead


In [3]:
sys.path.append('./third_party/CosyVoice/third_party/Matcha-TTS/')

cosyvoice = CosyVoice('pretrained_models/CosyVoice-300M-Instruct')

# zero_shot usage
prompt_speech_16k = load_wav('asset/zero_shot_lx_prompt.wav', 16000)
output = cosyvoice.inference_zero_shot(tts_text='收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。', 
                                       prompt_text='对立开来，但其实你可以想一想，在开车的过程中，是在交通规则的保护下驾驶更自由，还是随心所欲无视规则开车更自由呢？因此我们会发', 
                                       prompt_speech_16k=prompt_speech_16k)
torchaudio.save('zero_shot.wav', output['tts_speech'], 22050)

/opt/conda/envs/project_1_plus/lib/python3.8/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2024-08-09 11:29:38.536599556 [W:onnxruntime:, session_state.cc:1162 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-08-09 11:29:38.536611217 [W:onnxruntime:, session_state.cc:1164 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2024-08-09 11:29:38,680 WETEXT INFO found existing fst: /opt/conda/envs/project_1_plus/lib/python3.8/site-packages/tn/zh_tn_tagger.fst
2024-08-09 11:29:38,680 W

**SenseVoice**

In [1]:
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess

In [2]:
model_dir = 'pretrained_models/SenseVoiceSmall'

model = AutoModel(
    model=model_dir,
    trust_remote_code=True,
    remote_code="./third_party/SenseVoice-main/model.py",    
    vad_model="fsmn-vad",
    vad_kwargs={"max_single_segment_time": 30000},
    device="cuda:0",
)

You are using the latest version of funasr-1.1.4
Loading remote code successfully: ./third_party/SenseVoice-main/model.py


2024-08-09 15:07:40,047 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2024-08-09 15:07:40,048 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-08-09 15:07:40,135 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 750810b2d0c33f9be9cca0301d47437a and a total number of 980 components indexed


transformer is not installed, please install it if you want to use related modules


2024-08-09 15:07:40,361 - modelscope - WARNING - Using the master branch is fragile, please use it with caution!
2024-08-09 15:07:40,363 - modelscope - INFO - Use user-specified model revision: master


In [4]:
# en
res = model.generate(
    input=f"{model.model_path}/example/zh.mp3",
    cache={},
    language="auto",  # "zh", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)

rtf_avg: 0.009: 100%|██████████| 1/1 [00:00<00:00, 19.41it/s]
rtf_avg: 0.010, time_speech:  5.616, time_escape: 0.057: 100%|██████████| 1/1 [00:00<00:00, 10.53it/s]

开饭时间早上9点至下午5点。


In [1]:
import sys
sys.path.append('./third_party/SenseVoice-main/')
from model import SenseVoiceSmall

In [2]:
model_dir = 'pretrained_models/SenseVoiceSmall'

m_recogize, param_recogize = SenseVoiceSmall.from_pretrained(model=model_dir)

Loading remote code successfully: model


In [3]:
def process_audio(audio):
    global m_recogize, param_recogize
    print('Processing audio:', audio)
    res = m_recogize.inference(
            data_in=audio,
            language="zh", # "zn", "en", "yue", "ja", "ko", "nospeech"
            use_itn=False,**param_recogize)
    result = res[0][0]['text']
    return result

In [4]:
process_audio('/workspace/project_1_plus/asset/zero_shot_lx_prompt.wav')

Processing audio: /workspace/project_1_plus/asset/zero_shot_lx_prompt.wav


'<|zh|><|NEUTRAL|><|Speech|><|woitn|>对立开来但其实你可以想一想在开车的过程中是在交通规则的保护下驾驶更自由还是随心所欲无视规则开车更自由呢因此我们会发'

In [2]:
from ollama-python import OllamaClient

SyntaxError: invalid syntax (1207412447.py, line 1)

In [17]:
import requests
import json

def call_ollama_service(content):
    response = requests.post(
        "http://127.0.0.1:11434/api/chat",
        json={
            "model": "qwen:0.5b",
            "prompt": content,
            "format": "json", 
            "stream": False
        }
    )
    # print(response)
    # print(response.status_code, type(response.status_code))
    # if response.status_code == 200:
    #     return response.json()["response"]
    # else:
    #     return "Error: Unable to get response from Ollama service."
    print(response)
    if response.status_code == 200:
        response_content = ""
        for line in response.iter_lines():
            if line:
                response_content += json.loads(line)["message"]["content"]
        return response_content
    else:
        return f"Error: {response.status_code} - {response.text}"

In [19]:
def send_message_to_ollama(message, port=11434):
    url = f"http://localhost:{port}/api/chat"
    payload = {
        "model": "qwen:0.5b",
        "messages": [{"role": "user", "content": message}]
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        response_content = ""
        for line in response.iter_lines():
            if line:
                response_content += json.loads(line)["message"]["content"]
        return response_content
    else:
        return f"Error: {response.status_code} - {response.text}"

In [23]:
send_message_to_ollama('你好')

'您好！有什么可以帮助您的吗？'

In [24]:
send_message_to_ollama('你是谁？')

'我是阿里云研发的大规模语言模型，我叫通义千问。'

In [25]:
send_message_to_ollama('你刚才回答了什么问题？')

'我正在帮助用户解答问题。'